In [5]:
import pandas as pd
import os
from dotenv import load_dotenv
import json
import csv
import PIL
from transformers import pipeline
from tqdm import tqdm
import time

load_dotenv()

C:\Users\nello\AppData\Local\Temp\ipykernel_10580\1725253305.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\nello\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# 1) Continue from cleaning phase, showing the cardinalities of the cleaned dataset

In [2]:
multimodal_train_tsv_path = os.getenv('MULTIMODAL_TRAIN_CLEANED_WITH_CLASS_TSV')
multimodal_test_tsv_path = os.getenv('MULTIMODAL_TEST_CLEANED_WITH_CLASS_TSV')
multimodal_validation_tsv_path = os.getenv('MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV')

df_train = pd.read_csv(multimodal_train_tsv_path, sep='\t')
df_test = pd.read_csv(multimodal_test_tsv_path, sep='\t')
df_val = pd.read_csv(multimodal_validation_tsv_path, sep='\t')

train_counts = df_train['class'].value_counts()
test_counts = df_test['class'].value_counts()
val_counts = df_val['class'].value_counts()

print("Train Counts:")
print(train_counts)
print("\nTest Counts:")
print(test_counts)
print("\nValidation Counts:")
print(val_counts)

print("\n=> Total \"pristine\": "+ str(train_counts['pristine']+test_counts['pristine']+val_counts['pristine'])+" | Total \"fake\": "+str(train_counts['fake']+test_counts['fake']+val_counts['fake']))

Train Counts:
class
pristine    385872
fake        160261
Name: count, dtype: int64

Test Counts:
class
pristine    41567
fake        16820
Name: count, dtype: int64

Validation Counts:
class
pristine    40911
fake        16912
Name: count, dtype: int64

=> Total "pristine": 468350 | Total "fake": 193993


# 2) Select 250.000 pristine images to be captioned from train, test and val sets

I will select the first 20.000 from test, 20.000 from val, and 210.000 from train set.

In [3]:
# Select the first 210,000 pristine images from the train set
selected_train_pristine = df_train[df_train['class'] == 'pristine'].head(210000)

# Select the first 20,000 pristine images from the test set
selected_test_pristine = df_test[df_test['class'] == 'pristine'].head(20000)

# Select the first 20,000 pristine images from the validation set
selected_val_pristine = df_val[df_val['class'] == 'pristine'].head(20000)

print("Selected train pristine shape: ", selected_train_pristine.shape)
print("Selected test pristine shape: ", selected_test_pristine.shape)
print("Selected val pristine shape: ", selected_val_pristine.shape)


Selected train pristine shape:  (210000, 17)
Selected test pristine shape:  (20000, 17)
Selected val pristine shape:  (20000, 17)


Save the selected pristine of each set to csv

In [50]:
selected_train_pristine.to_csv("csv/selected_train_pristine.csv", index=False)
selected_test_pristine.to_csv("csv/selected_test_pristine.csv", index=False)
selected_val_pristine.to_csv("csv/selected_val_pristine.csv", index=False)

Define the ImageCaptioner class to be used to caption the pristine images from the sets

In [6]:
class ImageCaptioner:
    def __init__(self, device=0, model="Salesforce/blip-image-captioning-base"):
        self.device = device
        self.captioner = pipeline(
            "image-to-text",
            model=model,
            device=device,
        )

    def process_images(self, image_paths, batch_size=30):
        total_inference_time = 0
        result_dict = {"image_name": [], "caption": []}

        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i + batch_size]

            with tqdm(total=len(batch_paths),
                    desc=f"Processing Batch {i // batch_size + 1}/{len(image_paths) // batch_size}", unit="image") as pbar:
                start_time = time.time()

                try:
                    captions = self.captioner(batch_paths, max_new_tokens=100)
                except PIL.UnidentifiedImageError as ex:
                    error_entries = [{"path": path, "error": str(ex)} for path in batch_paths]
                    result_dict["image_name"].extend([entry["path"] for entry in error_entries])
                    result_dict["caption"].extend([entry["error"] for entry in error_entries])
                    pbar.update(len(batch_paths))
                    continue

                end_time = time.time()
                batch_inference_time = end_time - start_time
                total_inference_time += batch_inference_time

                for path, caption in zip(batch_paths, captions):
                    image_name = os.path.splitext(os.path.basename(path))[0]  # remove .jpg
                    result_dict["image_name"].append(image_name)
                    result_dict["caption"].append(caption[0]["generated_text"])
                    pbar.update(1)

        avg_time_per_image = total_inference_time / len(image_paths)
        print(f"\nTotal Inference Time: {total_inference_time:.2f} seconds")
        print(f"Avg Time Per Image Caption: {avg_time_per_image:.4f} seconds")

        result_df = pd.DataFrame(result_dict)
        return result_df


In [54]:
captioner = ImageCaptioner(model="Salesforce/blip-image-captioning-base")

Define the method to be called onto one set at a time, to caption all the images of a set (e.g. training set)

In [55]:
def caption_and_save_images(df, save_path):
    # Extract paths of selected pristine images
    pristine_paths = [os.path.join(DATASET_DIR, f"{image_name}.jpg") for image_name in df['id']]

    # Process and caption images
    generated_captions_df = captioner.process_images(image_paths=pristine_paths, batch_size=BATCH_SIZE)

    # Merge the original captions from df with the generated captions
    result_df = pd.merge(df, generated_captions_df, left_on="id", right_on="image_name", how="left")

    # Create a new DataFrame with 'id', 'original_caption', and 'caption'
    result_df = result_df[['id', 'clean_title', 'caption']]

    result_df = result_df.rename(columns={'clean_title': 'original_caption', 'caption': 'generated_caption'})
    result_df.to_csv(save_path, index=False, header=True, sep=',', encoding='utf-8')

Restore the csv of the selected pristine for each set

In [ ]:
selected_train_pristine_csv_path = os.getenv('SELECTED_TRAIN_PRISTINE_CSV_PATH')
selected_test_pristine_csv_path = os.getenv('SELECTED_TEST_PRISTINE_CSV_PATH')
selected_validation_pristine_csv_path = os.getenv('SELECTED_VAL_PRISTINE_CSV_PATH')

selected_train_pristine = pd.read_csv(selected_train_pristine_csv_path, sep=',')
selected_test_pristine = pd.read_csv(selected_test_pristine_csv_path, sep=',')
selected_val_pristine = pd.read_csv(selected_validation_pristine_csv_path, sep=',')

Caption 210.000 images from training set:

In [ ]:
caption_and_save_images(selected_train_pristine, "csv/training_pristine_captioned.csv")

Caption 20.000 images from test set:

In [56]:
caption_and_save_images(selected_test_pristine, "csv/test_pristine_captioned.csv")

Processing Batch 11/200:   0%|          | 0/100 [00:00<?, ?image/s]C:\Users\nello\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Processing Batch 15/200:   0%|          | 0/100 [00:00<?, ?image/s]C:\Users\nello\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing Batch 200/200: 100%|██████████| 100/100 [00:43<00:00,  2.31image/s]


Total Inference Time: 10052.93 seconds
Avg Time Per Image Caption: 0.5026 seconds


Caption 20.000 images from validation set:

In [57]:
caption_and_save_images(selected_val_pristine, "csv/validation_pristine_captioned.csv")

Processing Batch 1/200:   0%|          | 0/100 [00:00<?, ?image/s]C:\Users\nello\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Processing Batch 59/200:   0%|          | 0/100 [00:00<?, ?image/s]C:\Users\nello\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing Batch 153/200:   0%|          | 0/100 [00:00<?, ?image/s]